In [33]:
import numpy as np
import pandas as pd
import spacy
nlp = spacy.load("es_core_news_lg")
import nltk

#### Lectura Inicial del Corpus

In [34]:
df_canciones_raw = pd.read_csv("../../../data/data.csv")

In [35]:
df_canciones = df_canciones_raw.copy()
print(f'Registros: {len(df_canciones)}')
# Eliminación de NAs
df_canciones = df_canciones.dropna(subset=["Cancion", "Titulo"]).reset_index(drop=True)
print(f'Registros: {len(df_canciones)}')
# Eliminación de caracteres especiales
df_canciones['Cancion'] = df_canciones['Cancion'].replace({'\n':' ','\r':' '},regex=True) 
df_canciones['Cancion'] = df_canciones['Cancion'].replace({'\s+':' '},regex=True).str.strip()
# df_canciones["tokens"] = df_canciones["Cancion"].apply(tk.tokenize)
# df_canciones["size_tokens"] = df_canciones["tokens"].apply(len)
# df_canciones["normalize"] = df_canciones["Cancion"].apply(normalize_text)
# df_canciones = df_canciones[~(df_canciones["size_tokens"] < 10)].reset_index()
# df_canciones = df_canciones.groupby(["Genero"]).agg({"Cancion": lambda x: " ".join(x), "normalize": lambda x: " ".join(x), "Artista": "count", "size_tokens": [("mean", "mean"), ("min", "min"), ("max", "max")]}).reset_index()
# df_canciones.columns = ['_'.join(col).strip() for col in df_canciones.columns.values]
# df_canciones = df_canciones.rename(columns={"Genero_": "genero", "Cancion_<lambda>": "cancion", "normalize_<lambda>": "normalize", "Artista_count": "n_canciones"})
# df_canciones

Registros: 3068
Registros: 3065


In [36]:
print(df_canciones['Cancion'][0])

Llegó el momento, caen las murallas Va a comenzar a la única justa de las batallas No duele el golpe, no existe el miedo Quitate el polvo, ponte de pie y vuelves al ruedo Y la presión, se siente Espera en ti tu gente Ahora vamos por todo Y te acompaña la suerte Tsamina mina zangalewa Porqué esto es África Tsamina mina eh, eh Waka waka eh, eh Tsamina mina zangalewa Porqué esto es África Oye a tu dios y no estarás solo Llegaste aquí para brillar y lo tienes todo La hora se acerca, es el momento Vas a ganar a cada batalla, ya lo presiento Hay que empezar de cero Para tocar el cielo Ahora vamos por todo Y todos vamos por ellos Tsamina mina zangalewa Porqué esto es África Tsamina mina, eh eh Waka waka, eh eh Tsamina mina zangalewa Anawa, ah ah Tsamina mina, eh eh Waka waka, eh eh Tsamina mina zangalewa Porqué esto es África Ame la mejole biggi biggi mubbo wa A to Zet Asi tsu zala makyuni biggi biggi mubbo from east to west Bathi waka waka ma eh eh waka waka ma eh eh Zonke zizwe mazi buye Ca

### Funciones

#### Longitud de la canción

In [37]:
def calcular_lonigtud_cancion(letra_cancion):
    doc = nlp(letra_cancion)
    palabras = [token.text.lower() for token in doc if token.is_alpha and not token.is_stop and not token.is_punct]
    total_palabras = len(palabras)  # Número total de palabras
    return total_palabras

#### Diversidad Léxica

In [38]:
def calcular_diversidad_lexica(letra_cancion):
    doc = nlp(letra_cancion)
    palabras = [token.text.lower() for token in doc if token.is_alpha and not token.is_stop and not token.is_punct]
    palabras_unicas = len(set(palabras)) # Número de palabras únicas
    total_palabras = len(palabras)  # Número total de palabras
    if total_palabras == 0:
        return 0  
    else:
        diversidad_lexica = np.round(palabras_unicas / total_palabras,4)
        return diversidad_lexica

#### Número de Adjetivos, Verbos y Sustantivos

In [39]:
def porcentaje_verbos_unicos(letra_cancion):
    doc = nlp(letra_cancion)
    palabras = [token.text.lower() for token in doc if token.is_alpha and not token.is_stop and not token.is_punct]
    total_palabras = len(palabras)
    pos_objetivo = set([token.text.lower() for token in nlp(' '.join(palabras)) if token.pos_=='VERB'])
    total_pos_objetivo_unicos = len(pos_objetivo)
    if total_palabras == 0:
        return 0  
    else:
        return np.round(total_pos_objetivo_unicos / total_palabras,4)
    
def porcentaje_sustantivos_unicos(letra_cancion):
    doc = nlp(letra_cancion)
    palabras = [token.text.lower() for token in doc if token.is_alpha and not token.is_stop and not token.is_punct]
    total_palabras = len(palabras)
    pos_objetivo = set([token.text.lower() for token in nlp(' '.join(palabras)) if token.pos_=='NOUN'])
    total_pos_objetivo_unicos = len(pos_objetivo)
    if total_palabras == 0:
        return 0  
    else:
        return np.round(total_pos_objetivo_unicos / total_palabras,4)
    
def porcentaje_adjetivos_unicos(letra_cancion):
    doc = nlp(letra_cancion)
    palabras = [token.text.lower() for token in doc if token.is_alpha and not token.is_stop and not token.is_punct]
    total_palabras = len(palabras)
    pos_objetivo = set([token.text.lower() for token in nlp(' '.join(palabras)) if token.pos_=='ADJ'])
    total_pos_objetivo_unicos = len(pos_objetivo)
    if total_palabras == 0:
        return 0  
    else:
        return np.round(total_pos_objetivo_unicos / total_palabras,4)


## Transformación del Corpus

In [40]:
print(f'Número de canciones en el corpus: {len(df_canciones)}')
corpus_unido = df_canciones['Cancion'].str.cat(sep=' ')
n_palabras = len(df_canciones['Cancion'].str.cat(sep=' ').split(' '))
print(f'Número de "palabras" en el corpus: {n_palabras}')
print(f'Número de caracteres en el corpus: {len(corpus_unido)}')
df_features = df_canciones.loc[:,df_canciones.columns[:-1]].copy()

Número de canciones en el corpus: 3065
Número de "palabras" en el corpus: 921482
Número de caracteres en el corpus: 4690153


In [41]:
# Definición de los batches
batch_size = 500
batch_ranges = [range(i,min(i+batch_size,len(df_canciones))) for i in range(0,len(df_canciones),batch_size)]

### Features Lingüísticos

In [42]:
# Longitud y Diversidad Léxica
for i,br in enumerate(batch_ranges):
    batch = list(br)
    # Longitud Canción
    df_features.loc[batch,'longitud_cancion'] = df_canciones.loc[batch,'Cancion'].apply(calcular_lonigtud_cancion)
    # Diversidad Léxica
    df_features.loc[batch,'diversidad_lexica'] = df_canciones.loc[batch,'Cancion'].apply(calcular_diversidad_lexica)
    df_features.to_csv('features.csv')
    print(f'Batch {i+1} de {len(batch_ranges)} finalizado!')

Batch 1 de 7 finalizado!
Batch 2 de 7 finalizado!
Batch 3 de 7 finalizado!
Batch 4 de 7 finalizado!
Batch 5 de 7 finalizado!
Batch 6 de 7 finalizado!
Batch 7 de 7 finalizado!


In [43]:
# Verbos, Adjetivos y Sustantivos
for i,br in enumerate(batch_ranges):
    batch = list(br)
    # Longitud Canción
    df_features.loc[batch,'verbos_unicos'] = df_canciones.loc[batch,'Cancion'].apply(porcentaje_verbos_unicos)
    df_features.loc[batch,'adjetivos_unicos'] = df_canciones.loc[batch,'Cancion'].apply(porcentaje_adjetivos_unicos)
    df_features.loc[batch,'sustantivos_unicos'] = df_canciones.loc[batch,'Cancion'].apply(porcentaje_sustantivos_unicos)
    # # Diversidad Léxica
    # df_features.loc[batch,'diversidad_lexica'] = df_canciones.loc[batch,'Cancion'].apply(calcular_diversidad_lexica)
    df_features.to_csv('features.csv')
    print(f'Batch {i+1} de {len(batch_ranges)} finalizado!')

Batch 1 de 7 finalizado!
Batch 2 de 7 finalizado!
Batch 3 de 7 finalizado!
Batch 4 de 7 finalizado!
Batch 5 de 7 finalizado!
Batch 6 de 7 finalizado!
Batch 7 de 7 finalizado!


## Análisis de los Features